In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import nltk
import numpy as np # linear algebra
import pandas as pd
import fastai
from fastai import *
from fastai.text import *

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes',))
document = newsgroups_train.data

In [ ]:
document[0]

In [ ]:
df = pd.DataFrame({'label': newsgroups_train.target, 'texts': newsgroups_train.data})
df.head()

In [ ]:
print(df.shape)

In [ ]:
from pprint import pprint
pprint(list(newsgroups_train.target_names))

In [ ]:
newsgroups_train.target[:100]

In [ ]:
df['label'].value_counts()

## Data preprocessing

In [ ]:
# removing everything else except alphabels
df['texts'] = df['texts'].str.replace("[^a-zA-Z]", " ")

In [ ]:
df['texts'][0]

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

In [ ]:
# tokenisation
tokenized_doc = df['texts'].apply(lambda x: x.split())

tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['texts'] = detokenized_doc

In [ ]:
df['texts'][0]

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.2, random_state = 12)

In [ ]:
df_trn.shape, df_val.shape

In [ ]:
df.head()

## ULMfit model

In [ ]:
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [ ]:
data_lm.show_batch()

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(5, 1e-2)

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
# learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, 1e-3)

In [ ]:
learn.save_encoder('ft_enc1')

In [ ]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load_encoder('ft_enc_v1')

In [ ]:
# text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5 )

In [ ]:
learn.fit_one_cycle(2, 1e-2)

In [ ]:
learn.save('first')
learn.load('first')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)


In [ ]:
pd.crosstab(predictions, targets)

In [ ]:
learn.predict("Church is not always very helpful in relegious matters")

## Naive-bayes

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import numpy as np

## Converting text to vectors

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)
vectors.shape

In [ ]:
vectors.nnz / float(vectors.shape[0])

https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

### Filtering text into more real world

In [ ]:
newsgroups_test = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes',))

In [ ]:
vectors_test = vectorizer.transform(newsgroups_test.data)
clf = MultinomialNB(alpha=.01)

clf.fit(vectors, newsgroups_train.target)
pred = clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target, pred, average='macro')

In [ ]:
def show_top10(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(categories):
        top10 = np.argsort(classifier.coef_[i])[-10:]
        print("%s: %s" % (category, " ".join(feature_names[top10])))

show_top10(clf, vectorizer, newsgroups_train.target_names)